In [1]:
!pip3 install datasets
!pip3 install rouge_score
!pip3 install git+https://github.com/huggingface/transformers
!pip3 install sentencepiece
!pip3 install torch
!pip3 install transformers
!pip install --upgrade transformers
!pip install --upgrade datasets
!pip install tensorflow
!pip install ipywidgets
!pip install torch

You should consider upgrading via the '/Users/mw/.pyenv/versions/3.8.16/envs/nlu/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/mw/.pyenv/versions/3.8.16/envs/nlu/bin/python3.8 -m pip install --upgrade pip' command.
  Cloning https://github.com/huggingface/transformers to /private/var/folders/10/730vyzkx1tn7xdrft3tm6nvw0000gn/T/pip-req-build-hbq970_x
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /private/var/folders/10/730vyzkx1tn7xdrft3tm6nvw0000gn/T/pip-req-build-hbq970_x
  Resolved https://github.com/huggingface/transformers to commit 656e869a4523f6a0ce90b3aacbb05cc8fb5794bb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


You should consider upgrading via the '/Users/mw/.pyenv/versions/3.8.16/envs/nlu/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/mw/.pyenv/versions/3.8.16/envs/nlu/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/mw/.pyenv/versions/3.8.16/envs/nlu/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/mw/.pyenv/versions/3.8.16/envs/nlu/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/mw/.pyenv/versions/3.8.16/envs/nlu/bin/python3.8 -m pip install --upgrade pip' command.


You should consider upgrading via the '/Users/mw/.pyenv/versions/3.8.16/envs/nlu/bin/python3.8 -m pip install --upgrade pip' command.


You should consider upgrading via the '/Users/mw/.pyenv/versions/3.8.16/envs/nlu/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/mw/.pyenv/versions/3.8.16/envs/nlu/bin/python3.8 -m pip install --upgrade pip' command.


You should consider upgrading via the '/Users/mw/.pyenv/versions/3.8.16/envs/nlu/bin/python3.8 -m pip install --upgrade pip' command.


In [ ]:
# https://huggingface.co/yikuan8/Clinical-Longformer

In [3]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
from datasets import load_dataset, DatasetDict, Dataset
from transformers import DataCollatorForSeq2Seq, Trainer, TrainingArguments
from torch.utils.data import DataLoader
import torch
import os
import numpy as np
import pandas as pd

In [4]:
lr = 3e-5  # from paper
batch_size = 32
max_input_length = 4096
max_output_length = 1024

In [5]:
model_checkpoint = "yikuan8/Clinical-Longformer"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [8]:
model = AutoModelForMaskedLM.from_pretrained("yikuan8/Clinical-Longformer")

Some weights of the model checkpoint at yikuan8/Clinical-Longformer were not used when initializing LongformerForMaskedLM: ['longformer.embeddings.position_ids']
- This IS expected if you are initializing LongformerForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [10]:
def process_data_to_model_inputs(batch):
    # tokenize the inputs and labels
    inputs = tokenizer(
        batch["article"],
        padding="max_length",
        truncation=True,
        max_length=max_input_length,
    )
    outputs = tokenizer(
        batch["lay_summary"],
        padding="max_length",
        truncation=True,
        max_length=max_input_length,
    )

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask

    # create 0 global_attention_mask lists
    batch["global_attention_mask"] = len(batch["input_ids"]) * [
        [0 for _ in range(len(batch["input_ids"][0]))]
    ]

    # since above lists are references, the following line changes the 0 index for all samples
    batch["global_attention_mask"][0][0] = 1
    batch["labels"] = outputs.input_ids

    # We have to make sure that the PAD token is ignored
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in labels]
        for labels in batch["labels"]
    ]

    return batch

In [11]:
def load_article_dataset(dtype, filename, directory):
    path = os.path.join(directory, f'{dtype}/{filename}_{dtype}.jsonl')
    df = pd.read_json(path, lines=True, nrows=100)
    return Dataset.from_pandas(df)

def create_article_dataset_dict(filename, directory):
    dataset_types = ['train', 'val']
    datasets = {}
    for dtype in dataset_types:
        dataset = load_article_dataset(dtype, filename, directory)
        # dataset = dataset.map(lambda e: tokenizer(e['lay_summary'], truncation=True, 
        #                                                   padding='max_length', max_length=max_input_length),batched=True)
        # dataset['labels'] = dataset['input_ids']
        # dataset = dataset.map(lambda e: tokenizer(e['article'], truncation=True, 
        #                                                       padding='max_length', max_length=max_input_length), batched=True)
        # Apply the preprocess_function to the dataset
        dataset = dataset.map(
            process_data_to_model_inputs,
            batched=True,
            batch_size=batch_size,
            remove_columns=["article", "lay_summary", "headings"],
        )
        # Add the preprocessed dataset to the datasets dictionary
        dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'global_attention_mask', 'labels'])
        datasets[dtype] = dataset
    return DatasetDict(datasets)

In [12]:
filename = "eLife"
directory = "../data/task1_development/"
article_dataset = create_article_dataset_dict(filename, directory)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [13]:
num_train_epochs = 8
model_name = model_checkpoint.split("/")[-1]

In [14]:
from random import seed

def set_seed(seed_v: int = 42) -> None:
    np.random.seed(seed_v)
    seed(seed_v)
    torch.manual_seed(seed_v)
    torch.cuda.manual_seed(seed_v)
    # When running on the CuDNN backend, two further options must be set
    # torch.backends.cudnn.deterministic = True
    # torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed_v)
    print(f"Random seed set as {seed_v}")

In [15]:
print(torch.cuda.is_available())
torch.cuda.empty_cache()

False


In [16]:
set_seed(42)

Random seed set as 42


In [18]:
!pip install rouge

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
You should consider upgrading via the '/Users/mw/.pyenv/versions/3.8.16/envs/nlu/bin/python3.8 -m pip install --upgrade pip' command.


In [19]:
from rouge import Rouge

In [20]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    rouge = Rouge()
    scores = rouge.get_scores(predictions, labels, avg=True)
    return {
        "rouge1_f": scores["rouge-1"]["f"],
        "rouge2_f": scores["rouge-2"]["f"],
        "rougeL_f": scores["rouge-l"]["f"]
    }

In [21]:
args = TrainingArguments(
        output_dir='../tmp/',
        evaluation_strategy='epoch',
        save_strategy='epoch',
        learning_rate=lr,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        num_train_epochs=3,
        weight_decay=0.01,
        load_best_model_at_end=True,
        metric_for_best_model='rouge2_f',
        run_name=model_name,
    )

In [22]:
trainer = Trainer(
        model=model,
        args=args,
        train_dataset=article_dataset['train'],
        eval_dataset=article_dataset['val'],
        compute_metrics=compute_metrics,
    )

In [23]:
trainer.train()

/Users/mw/.pyenv/versions/3.8.16/envs/nlu/lib/python3.8/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


KeyboardInterrupt: 